## Add Operators

In [8]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import json

from src.lm_store import LMStore

In [5]:
# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Initialize LMStore
lm_store = LMStore(client)

# Load tools from JSON file
with open("data/tools.json", "r") as f:
    tools = json.load(f)

# Add all tools to LMStore
embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
added_count = lm_store.add(tools, embedding_model)

print(f"Added {added_count} tools to LMStore")
print(f"Total tools in store: {lm_store.count}")

lm_store.export("data/exported_tools.json")

Added 148 tools to LMStore
Total tools in store: 148


## Search Operator

In [11]:
# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
chat_model = os.getenv("AZURE_OPENAI_DEPLOYMENT")

# Initialize LMStore
lm_store = LMStore(client)

lm_store.load_from_file('data/exported_tools.json')

query = "How many indexes are supported in Workers Analytics Engine? Give an example using the Workers binding api."    

results, selection = lm_store.get(
    query=query,
    limit=5,
    llm_search=True,
    chat_model=chat_model,
    embedding_model=embedding_model
)

print("\nSearch results:")
for r in results:
    print(f"  {r.name}: {r.similarity_score:.4f}")

if selection:
    print(f"\nLLM selected: {selection}")


Search results:
  observability_values: 0.5368
  query_worker_observability: 0.5300
  observability_keys: 0.4915
  workers_builds_set_active_worker: 0.4014
  workers_list: 0.3819

LLM selected: observability_keys
